In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/NIR/data(markup 1, 2, 3, 5).csv" /content/data.csv

In [3]:
!cp "/content/drive/MyDrive/NIR/train.csv" /content/train.csv
!cp "/content/drive/MyDrive/NIR/eval.csv" /content/eval.csv

In [ ]:
import pandas as pd

train = pd.read_csv("/content/train.csv", sep=';', encoding='utf-8')
train.to_csv('train.csv', index=False, encoding='utf-8')

eval = pd.read_csv("/content/eval.csv", sep=';', encoding='utf-8')
eval.to_csv('eval.csv', index=False, encoding='utf-8')

#### Разделение набора данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("/content/data.csv")
# 500 в eval наборе данных
train, eval = train_test_split(data, test_size=500, random_state=3)

In [ ]:
train.to_csv('train.csv', index=False, encoding='utf-8')
eval.to_csv('eval.csv', index=False, encoding='utf-8')

##### Разметка набора данных train

In [ ]:
from IPython.display import clear_output 
from pprint import pprint

train['relevance'] = 0

# разметка 500 первых новостей train
for row_number, index in enumerate(train.index.tolist()[:500]):
  row = train.iloc[row_number]
  print(row['team1'], ' VS ', row['team2'])
  pprint(row['first_part_content'], width=150)
  print()
  pprint(row['part_body'], width=150)
  label = int(input())
  train.at[index, 'relevance'] = label
  clear_output()

In [ ]:
train['relevance'].unique()

array([1, 0])

In [ ]:
train.to_csv('train.csv', index=False, encoding='utf-8')

#### Разметка релевантных новостей с активным обучением

In [4]:
import os
import random
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [5]:
class CustomHuggingFaceDatasets(Dataset):
    """
    Support for `huggingface.datasets`: (https://github.com/huggingface/datasets).
    The purpose of this wrapper is to separate the labels from the rest of the sample information
    and make the dataset ready to be used by `baal.active.ActiveLearningDataset`.
    Args:
        dataset (Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter
            sequences.
    """

    def __init__(
        self,
        dataset,
        tokenizer=None,
        target_key: str = "label",
        input_key_1: str = "sentence1",
        input_key_2: str = "sentence1",
        max_seq_len: int = 512,
    ):
        self.dataset = dataset
        self.targets, self.text1, self.text2 = self.dataset[target_key], self.dataset[input_key_1], self.dataset[input_key_2]
        self.targets_list: List = np.unique(self.targets).tolist()
        self.input_ids, self.attention_masks = (
            self._tokenize(tokenizer, max_seq_len) if tokenizer else ([], [])
        )

    @property
    def num_classes(self):
        return len(self.targets_list)

    def _tokenize(self, tokenizer, max_seq_len):
        # For speed purposes, we should use fast tokenizers here, but that is up to the caller
        tokenized = tokenizer(
            self.text1,
            self.text2,
            add_special_tokens=True,
            max_length=max_seq_len,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
            truncation=True,
        )
        return tokenized["input_ids"], tokenized["attention_mask"]

    def label(self, idx: int, value: int):
        """Label the item.
        Args:
            idx: index to label
            value: Value to label the index.
        """
        self.targets[idx] = value

    def __len__(self):
        return len(self.text1)

    def __getitem__(self, idx):
        target = self.targets_list.index(self.targets[idx])

        return {
            "input_ids": self.input_ids[idx].flatten() if len(self.input_ids) > 0 else None,
            "inputs": self.text1[idx] + '\n' + self.text2[idx],
            "attention_mask": self.attention_masks[idx].flatten()
            if len(self.attention_masks) > 0
            else None,
            "label": torch.tensor(target, dtype=torch.long),
        }

def custom_active_huggingface_dataset(
    dataset,
    tokenizer=None,
    target_key: str = "label",
    input_key_1: str = "sentence1",
    input_key_2: str = "sentence2",
    max_seq_len: int = 512,
    **kwargs
):
    """
    Wrapping huggingface.datasets with baal.active.ActiveLearningDataset.
    Args:
        dataset (torch.utils.data.Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter sequences.
        kwargs (Dict): Parameters forwarded to 'ActiveLearningDataset'.
    Returns:
        an baal.active.ActiveLearningDataset object.
    """

    return ActiveLearningDataset(
        CustomHuggingFaceDatasets(dataset, tokenizer, target_key, input_key_1, input_key_2, max_seq_len), **kwargs
    )

Information on the hyperparms below

* epoch: Number of times you want to run and AL loop
* batch_size: The train and eval batch size for hf trainer arguments
* model: Hugging Face Model
* query_size: Number of samples you want to query at each AL iteration for labelling
* heuristic: The acquisition function/heuristic based on which you want to query the important samples
* iterations: The number of iterations you want to run for MCdropout to find the uncertanities
* shuffle_prop: Additional Noise to counter selection bias
* learning_epoch: Traing epochs for hugging face trainer

In [6]:
hyperparams = {
    "epoch": 4,
    "batch_size": 10,
    "model": "DeepPavlov/rubert-base-cased",
    "query_size": 50,
    "heuristic": "bald",
    "iterations": 10,
    "shuffle_prop": 0.05,
    "learning_epoch": 2,
}

In [7]:
# Check for CUDA
use_cuda = torch.cuda.is_available()
torch.backends.cudnn.benchmark = True

id2label = {0: "LABEL_0", 1: "LABEL_1"}
label2id = {"LABEL_0": 0, "LABEL_1": 1}

# Load Model
hf_model = AutoModelForSequenceClassification.from_pretrained(
    hyperparams["model"], num_labels=2, id2label=id2label, label2id=label2id
    )

# Setup tokenizer for model
tokenizer = AutoTokenizer.from_pretrained(hyperparams["model"])

# Enable dropouts for predictions
hf_model = patch_module(hf_model)

# Send model to device and setup cuda arguments
if use_cuda:
    hf_model.to("cuda:0")
    no_cuda = False
else:
    hf_model.to("cpu")
    no_cuda = True

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [8]:
# Define labels in your dataset
label_list = [0, 1]

# Load data from files
data = load_dataset("csv", data_files={'train': '/content/train.csv', 'eval': "/content/eval.csv"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-202cffd3752f27f5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def get_label_from_data(active_dataset, data, target, input1, input2, indexes) -> List[int]:
    """
    Get labels from the active dataset, this assumes that you have
    already labelled some samples in your initial dataset

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """

    labels = []

    # Now since you labelled points earlier now some part of pool has become train
    # so in order to get the pool indexes based on your 'original' data i.e
    # your raw_train_set. Make sure to user __pool_tp

    raw_data_idx = active_dataset._pool_to_oracle_index(indexes)

    for idx in raw_data_idx:
        #print(f"Adding labels for Raw data Index {idx} : {data[input1][idx]}")
        #print(f"Adding labels for Raw data Index {idx} : {data[input2][idx]}")

        label = data[target][idx]
        labels.append(label)
        #print(label)
        #print("\n")

    return labels

In [10]:
def get_label_human_oracle(active_dataset, indexes) -> List[int]:
    """
    Get labels from human oracle. During the AL loop some samples
    will go to the human labeller

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """
    # List for corresponding labels
    labels = []

    skipped = []
    
    for sample_idx, idx in enumerate(indexes):

        while True:
            try:
                print(idx)
                pprint(active_dataset.pool.__getitem__(idx)['inputs'], width=150)
                label = int(input())
            except ValueError:
                print("Sorry, I didn't understand that.")
                continue
            if label != -1 and label not in label_list:
                print(f"Allowed labels are {label_list}")
                continue
            if label == -1:
                print("Skipping this sample")
                skipped.append(sample_idx)
                break
            else:
                labels.append(label)
                break
        print("\n")

    indexes_upd = np.delete(indexes, skipped)

    return labels, indexes_upd

In [11]:
#small_data = data['train'].select(range(10_000))
small_data = data['train']

active_set = custom_active_huggingface_dataset(
    small_data,
    tokenizer=tokenizer,
    target_key='relevance',
    input_key_1='first_part_content',
    input_key_2="part_body"
)

active_set.can_label = True

# 500 первых объектов набора данных
label_from_data = get_label_from_data(active_set, small_data, 'relevance', 'first_part_content', 'part_body', range(500))

active_set.label(
    range(500),
    label_from_data,
)

valid_set = CustomHuggingFaceDatasets(data['eval'], tokenizer=tokenizer, target_key='relevance', input_key_1='first_part_content', input_key_2="part_body")

active_set, test_set = active_set, valid_set

In [12]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

# Model save checkpoint
save_checkpoint = 2

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [13]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")

    if (epoch != last_epoch):
      # MCdropout to gather uncertanities
      predictions = baal_trainer.predict_on_dataset(
          active_set.pool, iterations=hyperparams["iterations"]
      )
      print("\n")

      # Acquistion of the most informative and diverse samples based on BatchBALD
      top_uncertainty = heuristic(predictions)[: hyperparams.get("query_size", 1)]

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, top_uncertainty
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.511500,0.274425,0.928453
2,0.260500,0.215425,0.943396




[305-MainThread  ] [baal.transformers_trainer_wrapper:predict_on_dataset_generator:67] 2023-04-13T07:35:11.250012Z [info     ] Start Predict                  dataset=9500



100%|██████████| 950/950 [52:57<00:00,  3.34s/it]




9101
('Привет тем, кто читает эти строки. А тем, кто их поглощает, особый привет. Нас всех ожидает очередное московское дерби: "Локомотив" - ЦСКА. А '
 'мы-то его как ожидаем! Если железнодорожники со своими потерями определились до начала сезона, то чемпионы продолжают расставаться с лидерами '
 'прямо по ходу турнира. Вслед за Вагнером ушел и Мамаев. Но пока армейцы держатся на уровне "Локо" - те же 4 очка в копилке. Даже без Лава в обойме '
 'Слуцкий не выставил Думбия с первых минут. Автор дубля в прошлом туре будет ждать своего часа на скамье, пока в роли центрфорварда привычно '
 'попылит Ахмед Муса. А в защите у ЦСКА из-за травм основных латералей материализовался древнегаззаевский трехстолбовой блок из Игнашевича, '
 'обрамленного в массивную березуцкую раму. У "Локо" стартовый состав воссоздает ощущение какой-то давней благодати. Скорее всего, одновременным '
 'наличием в нем Чорлуки и Павлюченко. В остальном Кучук наделал еще кучу перестановок, в результате которых Шишкин о

...


100%|██████████| 940/940 [52:32<00:00,  3.35s/it]




3287
('Салют всем кальчонавтам! Судьба скудетто - здесь и сейчас: "Наполи" против "Ювентуса". Марадона приехал в Неаполь, показал кукиш налоговикам, '
 'пожелал "партенопейцам" удачной атаки и умотал фоткаться с теннисистами. А "Наполи" биться с чемпионом за трофей, которого они не брали со времен '
 'рукобожьего аргентинца. Если неаполитанцы хотят скудетто - им нужно обыгрывать туринцев. Без всяких компромиссов. 7 матчей без поражений против '
 '"Юве" на своем поле - хорошо, но нужно еще лучше. Ждем стартовый штурм хозяев. Но и "бьянконери" не будут отбывать номер. Нужно отзащищать свое '
 'преимущество над прямым конкурентом в шесть очков. Встреча с "Селтиком" в Лиге чемпионов очень скоро, но победа 3:0 в столице Шотландии позволяет '
 'команде Конте бросить основные силы на внутриитальянский фронт.  Понеслась! И уже удар Гамшика! Марек направляет мяч в том же направлении, что '
 'торчат и его волосы. Неблизкий промах. Джовинко попробовал обработать закидушку Вучинича, но не справи

 75%|███████▌  | 3/4 [3:11:19<1:03:35, 3815.83s/it]

Epoch,Training Loss,Validation Loss,F1
1,0.479500,0.208362,0.923077
2,0.140000,0.193609,0.962433


100%|██████████| 4/4 [3:14:21<00:00, 2915.31s/it]

In [14]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-130")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.17941583693027496,
 'eval_f1': 0.9642857142857143,
 'eval_runtime': 15.6907,
 'eval_samples_per_second': 31.866,
 'eval_steps_per_second': 3.187}

In [15]:
!cp -r "/content/checkpoint-130" "/content/drive/MyDrive/NIR/bald"

#### Разметка релевантных новостей со случайным обучением

In [17]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [18]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")
    if (epoch != last_epoch):
      point_idx_train = np.random.choice(len(active_set.pool) - 1, hyperparams.get("query_size", 1), replace=False)

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, point_idx_train
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # Save model
      #if epoch == save_checkpoint:
      #    save_model(baal_trainer)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,F1
1,0.349000,0.201728,0.950704
2,0.206100,0.206351,0.956370




7788
('Всем привет. Боевой английский тур продолжает матч бедовых лондонцев из "Вест Хэма" против действующих чемпионов из Мэдчестера. "Вест Хэм" - "Ман '
 'Сити". ... и все это на стадионе, когда-то называвшегося в честь несчастной Анны Болейн. Минута молчания. Балотелли добежал до Яаскелайнена, '
 'показал себя финскому вратарю, хоть мяч и не потрогал. Да, у "Сити" конечно разбавленный основной состав. На следующей неделе - судьбоносная игра '
 'в Лиге чемпионов с "Аяксом". ОГОГООО... Боковой арбитр отбирает у Нолана его гол - отличный удар по летящему мячу... Эх, не было же офсайда. Жаль. '
 'Молотобойцы держат мяч, за "Сити" пока довольно редкие контратаки. Пауза из-за травмы Нобла, мы снова и снова смотрим гол Нолана. Очень спорный '
 'момент, что, конечно, должно означать справедливость судейского решения. Хотя в принципе конечно хочется, чтобы такие эпизоды трактовали в пользу '
 'атакующих. Тяжелый на подъем удар Насри, Яаскелайнен играет отлично, а от добивания его хранит бо

 25%|██▌       | 1/4 [10:32<31:38, 632.83s/it]

Epoch,Training Loss,Validation Loss,F1
1,0.316200,0.248989,0.949153
2,0.308300,0.199669,0.944742


...

Epoch,Training Loss,Validation Loss,F1
1,0.399700,0.196064,0.953043
2,0.199500,0.224161,0.957143




8300
('Привет! Всю ночь снилась Лига чемпионов? Пора с этим прекращать. Лига Европы на дворе! БАТЭ (старые добрые знакомые) в Минске принимает "Эвертон" '
 '(с внешне не очень старым добрым знакомым в составе). Борисовский клуб успешен на белорусской арене, но первый матч Лиги Европы уступил "Бенфике" '
 '0:2. Тем не менее, Виктор Гончаренко считает, что игра у команды хорошая, а настроение и того прекрасней! Ливерпульцы стартовали в премьер-лиге '
 'катастрофическими 1:6 от "Арсенала", осели было на дне турнирной таблицы, но сейчас набрали хороший ход, давно не пропускают и исправно отправляют '
 'мячи в ворота соперников. Правда, в этот раз им придется обойтись без Саа, у которого дисквалификация, простывшего Родуэлла и охваченного '
 'проблемами колена Пиенара. Зато есть Билялетдинов, который штампует голевые передачи одну за другой. Мойес выразил удовлетворение игрой '
 'российского новичка, но в то же время не забыл сказать, что Динияру следует делать все немножко быстрее. Впроч

 75%|███████▌  | 3/4 [25:31<08:10, 490.77s/it]

Epoch,Training Loss,Validation Loss,F1
1,0.358600,0.199778,0.958855
2,0.214700,0.191871,0.959002


100%|██████████| 4/4 [28:29<00:00, 427.46s/it]

In [19]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-130")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.18700683116912842,
 'eval_f1': 0.9626998223801065,
 'eval_runtime': 16.3446,
 'eval_samples_per_second': 30.591,
 'eval_steps_per_second': 3.059}

In [21]:
!cp -r "/content/checkpoint-130" "/content/drive/MyDrive/NIR/random"